# Cutout data from fits files

## Imports

In [ ]:
import sys
import os
import pprint
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

root = "/Users/phdenzel/gleam"
sys.path.append(root)
import gleam.utils.colors as gcl

## Functions

In [ ]:
def onclick(event):
    global position
    global dot
    ix, iy = int(event.xdata), int(event.ydata)
    # print 'x, y = ({}, {})'.format(ix, iy)
    position = (ix, iy)
    dot.set_xdata(ix)
    dot.set_ydata(iy)
    plt.draw()

## Main

### Search for a fits file

In [ ]:
directories = !ls -d lensedquasars/*/
pprint.pprint(list(enumerate(directories)))

In [ ]:
files = !ls -d {directories[90]}*
files

In [ ]:
filename = os.path.basename(files[0])
filepath = files[0]
print(filepath)
print("Exists: {}".format(os.path.exists(filepath)))

### Read fits file and extract data

In [ ]:
hdu = fits.open(filepath)
print(hdu)
if 0:
    for h in hdu:
        if isinstance(h, fits.hdu.image.ImageHDU):
            hdu = h
            break
else:
    hdu = hdu[0]

position = tuple(s//2 for s in hdu.data.shape)

print(hdu.data.shape, position)
# print(repr(hhdu.header))

### Plot the data

In [ ]:
%matplotlib notebook

kw = dict(cmap=gcl.GLEAMcmaps.vilux, vmin=np.nanmin(hdu.data)*0.9, vmax=np.nanmax(hdu.data)*0.9)

fig, ax = plt.subplots()
dot, = ax.plot(position[0], position[1], marker='o', markersize=5, color=gcl.red)
cid = fig.canvas.mpl_connect('button_press_event', onclick)
pltdta = hdu.data[:]
# pltdta = np.log10(pltdta+1+np.abs(np.nanmin(pltdta)))
ax.imshow(pltdta, **kw)

In [ ]:
fig.canvas.mpl_disconnect(cid)
plt.close()
print("Cutout center @ {}".format(position))

In [ ]:
fs = fig.axes[0].viewLim
delta = int(0.5 + max(abs(fs.x1-fs.x0), abs(fs.y1-fs.y0)))
delta = int(np.ceil(delta / 99.0)) * 99
size = (delta, delta) if delta < hdu.data.shape[0] else hdu.data.shape
if size == hdu.data.shape:
    position = tuple(s//2 for s in hdu.data.shape)
print("position: {}".format(position), "size: {}".format(size))

### Ovewrite fits file and plot again

In [ ]:
wcs = WCS(hdu.header)
cutout = Cutout2D(hdu.data, position, size, wcs)
hdu.data = cutout.data
hdu.header.update(cutout.wcs.to_header())

In [ ]:
# show again for validation
%matplotlib notebook

# kw = dict(cmap=gcl.GLEAMcmaps.vilux, vmin=np.nanmin(hdu.data)*0.9, vmax=np.nanmax(hdu.data)*0.9)

fig, ax = plt.subplots()
pltdta = hdu.data[:]
# pltdta = np.log10(pltdta+1+np.abs(np.nanmin(pltdta)))
ax.imshow(pltdta, **kw)

### Save file

In [ ]:
savename = "{}.fits".format(os.path.dirname(filepath))
print("Saving {}...".format(savename))
hdu.writeto(savename, overwrite=True)

In [ ]:
!ls lensedquasars/*fits